In [1]:
import streamlit as st
import jsonlines
import random
import pandas as pd
import json
import fileinput
import numpy as np
from pprint import pprint

2022-05-22 08:26:04.387 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


In [2]:
#Importing Dataset
olafScholzJsonLines = jsonlines.open("OlafScholz.jl")

olafScholzTwitter = pd.read_json("OlafScholz.jl", lines = True)
print("Datensatzlänge: " + str(olafScholzTwitter.info()))

print(olafScholzTwitter.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   http_status   65 non-null     int64 
 1   account_name  65 non-null     object
 2   account_data  65 non-null     object
 3   params        65 non-null     object
 4   response      65 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.7+ KB
Datensatzlänge: None
   http_status account_name  \
0          200   OlafScholz   
1          200   OlafScholz   
2          200   OlafScholz   
3          200   OlafScholz   
4          200   OlafScholz   

                                        account_data  \
0  {'Name': 'Scholz, Olaf', 'Partei': 'SPD', 'id'...   
1  {'Name': 'Scholz, Olaf', 'Partei': 'SPD', 'id'...   
2  {'Name': 'Scholz, Olaf', 'Partei': 'SPD', 'id'...   
3  {'Name': 'Scholz, Olaf', 'Partei': 'SPD', 'id'...   
4  {'Name': 'Scholz, Olaf', 'Partei': 'SPD', 'id'...   

         

In [19]:
print("Response Metadaten: " + str(olafScholzTwitter["response"][0].keys()))
ResponseKeys = olafScholzTwitter["response"][0].keys()
print(ResponseKeys)
for Key in ResponseKeys:
    print(type(olafScholzTwitter["response"][0][Key]))
    #print("{}-Metadaten: ".format(Key) + str(olafScholzTwitter["response"][0][Key]))

Response Metadaten: dict_keys(['data', 'includes', 'meta'])
dict_keys(['data', 'includes', 'meta'])
<class 'list'>
<class 'dict'>
<class 'dict'>


In [4]:
#print(olafScholzTwitter["response"][0].len)
#print("---")
#Starting to investigate Dataset
#print(olafScholzTwitter.describe())
#print(type(olafScholzTwitter))
#print(olafScholzTwitter.keys())
#for key in olafScholzTwitter:
#    print("{} : {}".format(key, olafScholzTwitter[key]))
#    print("{}".format(key) + str(type(olafScholzTwitter[key])))
    

In [5]:
for line in olafScholzJsonLines:
    #print(line)
    keyResponse = line["response"]
    print("Response-Type:" + str(type(keyResponse)))
    print("Response-Keys:" + str(keyResponse.keys()))
    
    for key in keyResponse:
        print(type(keyResponse[key]))
        #pprint(keyResponse[key])
        #print("{}-Metadaten: ".format(key) + str(keyResponse[key].keys()))
    print("Includes-Metadaten: " + str(keyResponse["includes"].keys())+ "\n")
 
    print("Includes-Keys:" + str(keyResponse["includes"]["users"][0].keys())+"\n")
    #pprint("Example Includes:" +"\n" + str(keyResponse["includes"]["users"])+"\n")
        
    print("Meta-Metadaten: " + str(keyResponse["meta"].keys())+ "\n")
    data = keyResponse["data"]
    print("Type-Data: " + str(type(data))+ "\n")
    print(str(data[0]) + "\n")
    print("Type-Data[0]: " + str(type(data[0])) + "\n")
    print("data[0].keys(): " + str(data[0].keys()) + "\n")
    #print(dir(data[0].keys()))
    print("len(data[0].keys()): " + str(len(data[0].keys())) + "\n")
    break

Response-Type:<class 'dict'>
Response-Keys:dict_keys(['data', 'includes', 'meta'])
<class 'list'>
<class 'dict'>
<class 'dict'>
Includes-Metadaten: dict_keys(['users', 'tweets', 'media'])

Includes-Keys:dict_keys(['username', 'description', 'name', 'location', 'created_at', 'protected', 'id', 'entities', 'verified', 'public_metrics', 'profile_image_url', 'url'])

Meta-Metadaten: dict_keys(['newest_id', 'oldest_id', 'result_count', 'next_token'])

Type-Data: <class 'list'>

{'context_annotations': [{'domain': {'id': '10', 'name': 'Person', 'description': 'Named people in the world like Nelson Mandela'}, 'entity': {'id': '1439961464555466753', 'name': 'Olaf Scholz'}}, {'domain': {'id': '29', 'name': 'Events [Entity Service]', 'description': 'Entity Service related Events domain'}, 'entity': {'id': '1372917575357591554', 'name': 'Easter', 'description': 'this entity for all conversations for Easter all year '}}, {'domain': {'id': '35', 'name': 'Politician', 'description': 'Politicians in 

In [73]:
# Reply Dataframe
# Importing Data into final Dataframe for ML
replyDataFrame = pd.DataFrame(columns = ["target", "ids", "date", "flag", "user", "text"])
print(replyDataFrame.head())
#dir(replyDataFrame)
for line in olafScholzJsonLines:
    print(len(data))
    #pprint(data)
    #["target", "ids", "date", "flag", "user", "text"]
    newDataRow = [None, None, None, None, None, None]
    #print(line.keys())
    userName = line["account_name"]
    print("User Name: " + str(line["account_name"])+"\n")
    newDataRow[4] = userName
    keyResponse = line["response"]
    data = keyResponse["data"]


    for tweet in data:
        #print("Tweet-ID: " + tweet["id"])
        #print(str(tweet) + "\n")
        tweetTarget = None
        tweetID = tweet["id"]
        authorID = tweet["author_id"]
        tweetDate = tweet["created_at"]
        tweetFlag = None
        tweetText = tweet["text"]
        #tweetAuthor = tweet["context_annotations"]
        #tweetAuthor = tweet["context_annotations"]
        #tweetAuthor = userName
        #print(type(tweetAuthor))
        
        newDataRow[0] = tweetTarget
        newDataRow[1] = tweetID
        newDataRow[2] = tweetDate
        newDataRow[3] = tweetFlag
        #newDataRow[4] = tweetAuthor
        newDataRow[5] = tweetText
        #pprint(newDataRow)
        replyDataFrame.loc[len(replyDataFrame)] = newDataRow
        #tempDataFrame = pd.DataFrame(newDataRow, columns["target", "ids", "date", "flag", "user", "text"])
        #replyDataFrame = replyDataFrame.append(tempDataFame, ignore_index = True)
        #replyDataFrame = replyDataFrame.append(pd.DataFrame(newDataRow, columns = ["target", "ids", "date", "flag", "user", "text"]),ignore_index = True)
        #break
    pprint("Finale Dataframe \n" + str(replyDataFrame.head()))
    break

    


Empty DataFrame
Columns: [target, ids, date, flag, user, text]
Index: []
100
User Name: OlafScholz

('Finale Dataframe \n'
 '  target         ids                      date  flag        user  \\\n'
 '0   None  2909376466  2009-07-29T13:29:18.000Z  None  OlafScholz   \n'
 '1   None  2908973765  2009-07-29T12:58:36.000Z  None  OlafScholz   \n'
 '2   None  2889296362  2009-07-28T12:56:04.000Z  None  OlafScholz   \n'
 '3   None  2867104105  2009-07-27T07:50:35.000Z  None  OlafScholz   \n'
 '4   None  2854509309  2009-07-26T15:58:36.000Z  None  OlafScholz   \n'
 '\n'
 '                                                text  \n'
 '0  Auftakt des Ideenwettbewerbs "Gute Arbeit für ...  \n'
 '1  Interview mit dem RBB Inforadio: Olaf Scholz h...  \n'
 '2  "Ich glaube, dass wir mit einem blauen Auge da...  \n'
 '3  Jetzt auch bei Facebook: http://tinyurl.com/nj...  \n'
 '4  Interview mit der Bild am Sonntag: "Wir haben ...  ')


In [52]:
"""
# OlafTwitter Dataframe
# Importing Data into final Dataframe for ML
olafTwitterDataframe = pd.DataFrame(columns = ["target", "ids", "date", "flag", "user", "text"])
print(olafTwitterDataframe.head())
#dir(olafTwitterDataframe)
for line in olafScholzJsonLines:
    #["target", "ids", "date", "flag", "user", "text"]
    print(line.keys())
    pprint(line)
    break
    newDataRow = [None, None, None, None, None, None]
    
    keyResponse = line["response"]
    data = keyResponse["data"]
    print(len(data))
    for tweet in data:
        #print("Tweet-ID: " + tweet["id"])
        print(str(tweet) + "\n")
        tweetTarget = None
        tweetID = tweet["id"]
        authorID = tweet["author_id"]
        tweetDate = tweet["created_at"]
        tweetFlag = None
        tweetText = tweet["text"]
        #tweetAuthor = tweet["context_annotations"]
        #tweetAuthor = tweet["context_annotations"]
        tweetAuthor = None
        #print(type(tweetAuthor))
        
        
        newDataRow[0] = tweetTarget
        newDataRow[1] = tweetID
        newDataRow[2] = tweetDate
        newDataRow[3] = tweetFlag
        newDataRow[4] = tweetAuthor
        newDataRow[5] = tweetText
        pprint(newDataRow)
        olafTwitterDataframe.loc[len(olafTwitterDataframe)] = newDataRow
        #tempDataFrame = pd.DataFrame(newDataRow, columns["target", "ids", "date", "flag", "user", "text"])
        #olafTwitterDataframe = olafTwitterDataframe.append(tempDataFame, ignore_index = True)
        #olafTwitterDataframe = olafTwitterDataframe.append(pd.DataFrame(newDataRow, columns = ["target", "ids", "date", "flag", "user", "text"]),ignore_index = True)
        break
    #pprint("Finale Dataframe \n" + str(olafTwitterDataframe.head()))
    break
"""

Empty DataFrame
Columns: [target, ids, date, flag, user, text]
Index: []
dict_keys(['http_status', 'account_name', 'account_data', 'params', 'response'])
{'account_data': {'Name': 'Scholz, Olaf',
                  'Partei': 'SPD',
                  'id': 38150247,
                  'screen_name': 'OlafScholz',
                  't0': '20211020-103923.json',
                  'value': 'Scholz, Olaf'},
 'account_name': 'OlafScholz',
 'http_status': 200,
 'params': [['query', 'from:OlafScholz'],
            ['max_results', '100'],
            ['start_time', '2007-01-01T00:00:00Z'],
            ['end_time', '2022-04-18T11:59:59Z'],
            ['tweet.fields',
             'attachments,author_id,context_annotations,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang,possibly_sensitive,public_metrics,referenced_tweets,reply_settings,source,text,withheld'],
            ['expansions',
             'author_id,referenced_tweets.id,referenced_tweets.id.author_id,entities.mentions